Assumption: we have an input text of 6 words, and each word is represented by a 512 dimension vector.

In this case, the $sequence = 6$ and $d_{model} = 512$.

Therefore:

$\text{Input matrix} (sequence, d_{model}) = (6, 512)$

Let's assume that our words are:

$\text{A, B, C, D, E, F}$

Then, the illustration of the input matrix looks like this:

$$
\text{Input matrix} = \begin{bmatrix}
\text{A}, & \text{B}, & \text{C}, & \text{D}, & \text{E}, & \text{F} \\
\end{bmatrix}
$$


Let's assume that each word has the following 512 columns of numerical representation (basically acting like 512 dimensions):

$$

\text{word} = \begin{bmatrix}
W_{0},  & W_{1}, & W_{2}, & \dots & W_{511} \\
\end{bmatrix}
$$

Note that each $W_{i}$ here is a number.

With this background, the input matrix (let's call it $W$) in an expanded form should look like this:

$$
W = \begin{bmatrix}
\begin{bmatrix} \text{A}_0, & \text{A}_1, & \text{A}_2, & \dots, & \text{A}_{511} \end{bmatrix} \\
\begin{bmatrix} \text{B}_0, & \text{B}_1, & \text{B}_2, & \dots, & \text{B}_{511} \end{bmatrix} \\
\begin{bmatrix} \text{C}_0, & \text{C}_1, & \text{C}_2, & \dots, & \text{C}_{511} \end{bmatrix} \\
\begin{bmatrix} \text{D}_0, & \text{D}_1, & \text{D}_2, & \dots, & \text{D}_{511} \end{bmatrix} \\
\begin{bmatrix} \text{E}_0, & \text{E}_1, & \text{E}_2, & \dots, & \text{E}_{511} \end{bmatrix} \\
\begin{bmatrix} \text{F}_0, & \text{F}_1, & \text{F}_2, & \dots, & \text{F}_{511} \end{bmatrix}
\end{bmatrix}
$$

The shape, as we can clearly see, is $(6, 512)$.

Also, the transpose of this looks like this:

$$
W^\text{T} = \begin{bmatrix}
\begin{bmatrix} \text{A}_0, & \text{B}_0, & \text{C}_0, & \text{D}_0, & \text{E}_0, & \text{F}_0 \end{bmatrix} \\
\begin{bmatrix} \text{A}_1, & \text{B}_1, & \text{C}_1, & \text{D}_1, & \text{E}_1, & \text{F}_1 \end{bmatrix} \\
\begin{bmatrix} \text{A}_2, & \text{B}_2, & \text{C}_2, & \text{D}_2, & \text{E}_2, & \text{F}_2 \end{bmatrix} \\
\begin{bmatrix} \text{A}_3, & \text{B}_3, & \text{C}_3, & \text{D}_3, & \text{E}_3, & \text{F}_3 \end{bmatrix} \\
\vdots \\
\begin{bmatrix} \text{A}_{511}, & \text{B}_{511}, & \text{C}_{511}, & \text{D}_{511}, & \text{E}_{511}, & \text{F}_{511} \end{bmatrix}
\end{bmatrix}
$$

The shape of this is $(512, 6)$.

In [45]:
import torch
import torch.nn as nn

In [31]:
torch.manual_seed(40)
input_matrix = torch.randn(6, 512)
input_matrix.shape, input_matrix

(torch.Size([6, 512]),
 tensor([[-0.2367,  1.8109,  0.1966,  ..., -0.6320,  0.3352,  0.3928],
         [ 0.0783,  0.5694, -0.6083,  ...,  0.3377,  0.9911, -1.0636],
         [ 0.0525, -0.4094, -0.7481,  ...,  0.7475, -1.0518, -0.2402],
         [-0.7422,  0.5986,  1.1324,  ...,  0.6658, -1.9029,  0.3874],
         [ 0.3757, -0.0370,  0.0536,  ...,  1.3480, -0.3502,  1.4096],
         [ 0.3379,  0.6135,  0.3060,  ..., -0.1643, -1.4237,  1.1242]]))

In [44]:
input_matrix.T.shape, input_matrix.T

(torch.Size([512, 6]),
 tensor([[-0.2367,  0.0783,  0.0525, -0.7422,  0.3757,  0.3379],
         [ 1.8109,  0.5694, -0.4094,  0.5986, -0.0370,  0.6135],
         [ 0.1966, -0.6083, -0.7481,  1.1324,  0.0536,  0.3060],
         ...,
         [-0.6320,  0.3377,  0.7475,  0.6658,  1.3480, -0.1643],
         [ 0.3352,  0.9911, -1.0518, -1.9029, -0.3502, -1.4237],
         [ 0.3928, -1.0636, -0.2402,  0.3874,  1.4096,  1.1242]]))

In [9]:
transpose_dotproduct = input_matrix @ input_matrix.T
transpose_dotproduct.shape, transpose_dotproduct

(torch.Size([6, 6]),
 tensor([[503.2782,  30.9015, -25.3849,  13.0093, -29.7297,  20.1604],
         [ 30.9015, 536.9385, -23.3817, -70.6838,  -8.7096, -19.2283],
         [-25.3849, -23.3817, 484.9792,  24.6444, -29.3236,  -0.9737],
         [ 13.0093, -70.6838,  24.6444, 538.0654, -25.3541,  -8.6208],
         [-29.7297,  -8.7096, -29.3236, -25.3541, 453.2016,  -5.3324],
         [ 20.1604, -19.2283,  -0.9737,  -8.6208,  -5.3324, 539.1165]]))

In [23]:
A = input_matrix[0][:] # first row
A @ A.T # basically dot product of A and A^T

tensor(503.2783)

## Positional Encoding

Formulas for positional encoding in the original transformer paper are:

$$
PE(pos, 2i) = sin(pos /  10000^{2i/d_{model}}) \quad \leftarrow \text{(for even i)}
$$

$$
PE(pos, 2i+1) = cos(pos / 10000^{2i/d_{model}}) \quad \leftarrow \text{(for odd i)}
$$

where $pos$ is the position of the word in the sentence, and $i$ is the dimension index (starting from 0 to $d_{model} - 1$).









## Multi-Head Attention

Formula for multi-head attention in the original transformer paper is:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \text{head}_2, \dots, \text{head}_h)W^\text{O}
$$

where:

$
\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)
$

where $W_i^Q, W_i^K, W_i^V$ are the weights for the $i$-th head.

and:

$
W^\text{O} = \begin{bmatrix}
W_1^\text{O}, & W_2^\text{O}, & \dots, & W_h^\text{O}
\end{bmatrix}
$

